### _keras_tuner.SklearnTuner_ class performs cross-validated hyperparameter search for Scikit-learn models.

In [1]:
import keras_tuner as kt
from tensorflow import keras

from sklearn import metrics
from sklearn import datasets
from sklearn import model_selection
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier

2022-06-30 18:08:53.810598: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-30 18:08:53.810682: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def build_model(hp):
    model_type = hp.Choice('model_type', ['random_forest', 'ridge'])
    if model_type == 'random_forest':
        model = RandomForestClassifier(
            n_estimators = hp.Int('n_estimators', 10, 50, step = 10),
            max_depth = hp.Int('max_depth', 3, 10))
        
    else:
        model = RidgeClassifier(
            alpha = hp.Float('alpha', 1e-3, 1, sampling = 'log'))
    
    return model

In [3]:
tuner = kt.tuners.SklearnTuner(
    oracle = kt.oracles.BayesianOptimizationOracle(
        objective = kt.Objective('score', 'max'),
        max_trials = 10),
    hypermodel = build_model,
    scoring = metrics.make_scorer(metrics.accuracy_score),
    directory = '.',
    project_name = 'my_project')

In [4]:
X, y = datasets.load_iris(return_X_y = True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)

In [5]:
tuner.search(X_train, y_train)

Trial 10 Complete [00h 00m 00s]
score: 0.8166666666666667

Best score So Far: 0.9416666666666667
Total elapsed time: 00h 00m 02s
INFO:tensorflow:Oracle triggered exit


In [6]:
best_model = tuner.get_best_models(num_models = 2)

In [7]:
best_model

[RandomForestClassifier(max_depth=5, n_estimators=10),
 RidgeClassifier(alpha=0.31072014785280966)]